In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
####################################################################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 
# 8/23/23 We want to add:
#     Take into account the RELATIVE STIM LOCATION within every barrel. Does velocity depend on distance from stimulating location?
#            New column: distance (um) from stim loc to relevant barrel pair boundary (1:2)
#            New column: distance (um) from stim loc to relevant barrel pair boundary (2:3)
#
#     METHOD
#         To calculate stim location to boundary, take these files produced by MovieMaker2.ipynb:
#             electrode.dat
#             corners_layer.dat
#             corners_barrel.dat
#         Calculate: Distance to lower barrel, distance to higher barrel,
#                    Distance to lower layer, distance to higher layer
#                       (if applicable for each)
####################################################################################

In [2]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
cell_type_prefix = 'scnn1a'
csv_prefix = cell_type_prefix + '_all_slices.csv'
csv_columns = ['Date',
             'Slice/Loc/Rec',
             'Stim Location',
             'L4 barrel 1 Latency',
             'L4 barrel 2 Latency',
             'L4 barrel 3 Latency',
             'L4 barrel 1 SNR',
             'L4 barrel 2 SNR',
             'L4 barrel 3 SNR',
             'ROI jpg',
              'Age',
              'Infusion',
              'Sex',
              'Stim (uA)',
              'Delta latency 1 (barrel 1 - barrel 2)',
              'Delta latency 2 (barrel 3 - barrel 2)']

enable_photoZ_interaction = False
initialize_photoZ = False
snr_cutoff_signal = 5.0

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/" + cell_type_prefix +"_catalog.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,ROI jpg,Age,Infusion,Sex,Stim (uA),Delta latency 1 (barrel 1 - barrel 2),Delta latency 2 (barrel 3 - barrel 2),abs(delta 1),abs(delta 2)
0,2/28/2023,02_01_01,NaN,L5,50.3009,50.8406,49.3457,2.940390,10.630900,6.484390,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,-1.4949,NaN,1.4949
1,2/28/2023,02_02_02,NaN,L5,49.9882,51.3212,51.1805,2.045210,5.142090,2.921330,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,NaN,NaN,NaN
2,2/28/2023,03_01_01,NaN,L2/3,52.5719,51.2136,52.9013,9.405090,10.237200,5.780930,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.3583,1.6877,1.3583,1.6877
3,2/28/2023,03_02_02,NaN,L2/3,51.8399,50.0310,50.7536,14.738200,35.400200,21.883200,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.8089,0.7226,1.8089,0.7226
4,2/28/2023,06_01_01,NaN,L5,51.1367,51.0128,50.4326,5.726470,13.058300,13.063500,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,0.1239,-0.5802,0.1239,0.5802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,6/28/2023,02_04_03,Posterior,L4,58.3278,94.0000,94.0000,1.284740,0.000000,0.000000,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN
86,6/28/2023,03_01_10,Posterior,L4,94.0000,51.1749,51.3676,0.000000,0.263371,1.110580,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN
87,6/28/2023,03_02_03,Posterior,L4,58.9868,58.3472,94.0000,0.077615,0.514122,0.000000,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN
88,6/28/2023,03_03_01,Posterior,L4,50.7398,94.0000,94.0000,0.862532,0.000000,0.000000,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN


In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [4]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'slice' in f[-17:] and f.endswith('_rois.dat'):
            slice_no = int(f.split("_rois.dat")[0][-1])
            roi_files[slice_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
            
        # reach into analysis##_##_## subdir (created by MovieMaker2)
        analysis_subdir = subdir + '/analysis' + rec_id
        corners_barrel_file = analysis_subdir + "/corners_barrel.dat"
        corners_layer_file = analysis_subdir + "/corners_layer.dat"
        electrode_file = analysis_subdir + "/electrode.dat"
        
        # read in barrel and layer axes data
        barrel_layer_axes = []
        for la_file in [corners_barrel_file, corners_layer_file]:
            with open(la_file, 'r') as f:
                lines = f.readlines() 
            corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
            layer_axes = LayerAxes(corners)
            laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
            barrel_layer_axes.append(laminar_axis)
            barrel_layer_axes.append(laminar_axis_2)
        
        # open electrode as a single integer (its diode number) in variable stim_pt
        with open(electrode_file, 'r') as f:
            lines = f.readlines()
        stim_pt = int(lines[-1]) # last line is always electrode location
        aux_obj = LaminarROI([stim_pt]).get_points()
        stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
        print("Electrode stim point:", stim_pt)
        
        # compute perpendicular distances between each of the 4 lines and the electrode point
        perp_dists = []
        for line_obj in barrel_layer_axes:
            print(line_obj.get_line_repr(), line_obj.get_line_formula())
            perp_dists.append(line_obj.get_distance_to_point(stim_pt))
        print(perp_dists)
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = roi_files[slice_no]
        print("\n\nscraping data from ", zda_file, "using", roi_file, "on date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(roi_file)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)

        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from cell_type_prefix_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        
        # open these various files and store them to the pandas dataframe
        # The first three ROIs of each should correspond to the 3 desired barrel ROIs
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
        print(new_row)
        df = df.append(new_row, ignore_index=True)


edge_pts [[18, 7], [35, 9]] axis_pts [[22, 68], [55, 71]]
edge_pts [[76, 37], [75, 66]] axis_pts [[8, 33], [5, 59]]
Electrode stim point: [54, 50]
[[22, 68], [18, 7]] (61, -4, -1070)
[[55, 71], [35, 9]] (62, -20, -1990)
[[8, 33], [76, 37]] (-4, 68, -2212)
[[5, 59], [75, 66]] (-7, 70, -4095)
[33.10922073570213, 5.495349760617375, 14.269451334207293, 13.831016943918849]


scraping data from  01_01_03.zda using slice1_rois.dat on date: 6/26/2023
{'Date': '6/26/2023', 'Slice/Loc/Rec': '01_01_03', 'Age': 81, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 58.4733, 'L4 barrel 2 Latency': 51.3893, 'L4 barrel 3 Latency': 51.6462, 'L4 barrel 1 SNR': 0.408783, 'L4 barrel 2 SNR': 1.71941, 'L4 barrel 3 SNR': 1.49933, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-26-23\\selected_zda/ROIs_01_01_03.jpg'}
edge_pts [[7, 20], [28, 8]] axis_pts [[24, 67], [53, 69]]
edge_pts [[76, 33], [74, 60]] axis_pts [[9, 27], [8, 54]]
Electrode stim point: [25, 49]
[[24

{'Date': '6/26/2023', 'Slice/Loc/Rec': '03_05_01', 'Age': 81, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 53.4246, 'L4 barrel 2 Latency': 51.2068, 'L4 barrel 3 Latency': 52.8674, 'L4 barrel 1 SNR': 1.8833, 'L4 barrel 2 SNR': 5.59729, 'L4 barrel 3 SNR': 2.5859, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-26-23\\selected_zda/ROIs_03_05_01.jpg'}
edge_pts [[78, 24], [75, 56]] axis_pts [[7, 49], [50, 72]]
edge_pts [[74, 72], [46, 72]] axis_pts [[15, 9], [8, 35]]
Electrode stim point: [14, 44]
[[7, 49], [78, 24]] (25, 71, -3654)
[[50, 72], [75, 56]] (16, 25, -2600)
[[15, 9], [74, 72]] (-63, 59, 414)
[[8, 35], [46, 72]] (-37, 38, -1034)
[2.3913008598521612, 42.98953228976354, 24.65434603723023, 2.2625405940151215]


scraping data from  03_06_01.zda using slice3_rois.dat on date: 6/26/2023
{'Date': '6/26/2023', 'Slice/Loc/Rec': '03_06_01', 'Age': 81, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 51.1487, 'L4 barrel

{'Date': '6/28/2023', 'Slice/Loc/Rec': '02_03_04', 'Age': 83, 'Sex': 'F', 'Infusion': 'GABAzine', 'Stim (uA)': '200', 'L4 barrel 1 Latency': 52.6554, 'L4 barrel 2 Latency': 51.2084, 'L4 barrel 3 Latency': 52.1305, 'L4 barrel 1 SNR': 0.847673, 'L4 barrel 2 SNR': 2.49984, 'L4 barrel 3 SNR': 1.58351, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-28-23\\selected_zda/ROIs_02_03_04.jpg'}
edge_pts [[6, 54], [8, 14]] axis_pts [[63, 69], [73, 41]]
edge_pts [[58, 10], [77, 37]] axis_pts [[10, 64], [51, 70]]
Electrode stim point: [31, 61]
[[63, 69], [6, 54]] (15, -57, 2988)
[[73, 41], [8, 14]] (27, -65, 694)
[[10, 64], [58, 10]] (54, 48, -3612)
[[51, 70], [77, 37]] (33, 26, -3503)
[0.40718927884743006, 34.5814001126401, 13.702504175867096, 21.27968348398233]


scraping data from  02_03_10.zda using slice2_rois.dat on date: 6/28/2023
{'Date': '6/28/2023', 'Slice/Loc/Rec': '02_03_10', 'Age': 83, 'Sex': 'F', 'Infusion': 'GABAzine', 'Stim (uA)': '200', 'L4 barrel 1 Latency': 9

In [5]:
# compute statistics on barrel pairs:
#    1) whether delta latency should be considered based on SNR
#    2) Delta latency between L4 barrels (b1 - b2, b3 - b2)
df['Delta latency 1 (barrel 1 - barrel 2)'] = df['L4 barrel 1 Latency'] - df['L4 barrel 2 Latency']
df['Delta latency 2 (barrel 3 - barrel 2)'] = df['L4 barrel 3 Latency'] - df['L4 barrel 2 Latency']

# if either barrel 1 or barrel 2 has low SNR, don't compute the delta latency between these two
df.loc[(df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 1 (barrel 1 - barrel 2)'] = None

# same for barrel 2 or barrel 3
df.loc[(df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 2 (barrel 3 - barrel 2)'] = None

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                               (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 1 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 3 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[(df2['L4 barrel 1 SNR'] > snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_cross += len(df2[(df2['L4 barrel 2 SNR'] > snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] < snr_cutoff_signal)])
    n_barrel_none += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] < snr_cutoff_signal)])
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
df['abs(delta 1)'] = np.abs(df['Delta latency 1 (barrel 1 - barrel 2)'])
df['abs(delta 2)'] = np.abs(df['Delta latency 2 (barrel 3 - barrel 2)'])



for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    dfd1 = df2['abs(delta 1)'].dropna()
    dfd2 = df2['abs(delta 2)'].dropna()

    print("Latency between barrel pairs in", stim_loc)
    dfds = np.concatenate([dfd1, dfd2])
    print("\t", 
          str(np.mean(dfds))[:5], 
          "+/-", 
          str(np.std(dfds))[:5], 
          "(n =", 
          str(len(dfds)) + ")")
    print("\t", dfds)

In [ ]:
# write the pandas dataframe back to csv
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [ ]:
df